<a href="https://colab.research.google.com/github/mralamdari/NLP-Text_Classification/blob/main/NLP_Text_Classification_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install ktrain
!pip install pytorch_pretrained_bert pytorch-nlp
!pip install tqdm
!pip install lime

In [21]:
import io
import os
import re
import sys
import bs4
import tqdm
import wget
import nltk
import shap
import torch
import ktrain
import tarfile
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from ktrain import text
from sklearn import base
from zipfile import ZipFile
from sklearn import metrics
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import model_selection
warnings.filterwarnings('ignore')

# Data
###IMDB Dataset

In [ ]:
!wget -P DATAPATH http://nlp.stanford.edu/data/glove.6B.zip
!unzip DATAPATH/glove.6B.zip -d DATAPATH/glove.6B

!wget -P DATAPATH http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xvf DATAPATH/aclImdb_v1.tar.gz -C DATAPATH
BASE_DIR = 'Data'

In [ ]:
BASE_DIR = '/content/DATAPATH'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TRAIN_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/train')
TEST_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/test')

In [ ]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [ ]:
def get_data(data_dir):
  texts = []
  labels_index = {'pos': 1, 'neg': 0}
  labels = []
  for name in sorted(os.listdir(data_dir)):
    path = os.path.join(data_dir, name)
    if os.path.isdir(path):
      if name=='pos' or name=='neg':
        label_id = labels_index[name]
        for fname in sorted(os.listdir(path)):
          fpath = os.path.join(path, fname)
          text = open(fpath, encoding='utf8').read()
          texts.append(text)
          labels.append(label_id)
  return texts, labels

In [ ]:
train_texts, train_labels = get_data(TRAIN_DATA_DIR)
test_texts, test_labels = get_data(TEST_DATA_DIR)

labels_index = {'pos': 1, 'neg': 0}

In [ ]:
train_texts[20]

"The characters are unlikeable and the script is awful. It's a waste of the talents of Deneuve and Auteuil."

#Models

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index

In [ ]:
np.asarray(train_sequences).shape

(25000,)

In [ ]:
print(len(train_sequences[0]))
print(train_sequences[0])

109
[62, 4, 3, 129, 34, 44, 7576, 1414, 15, 3, 4252, 514, 43, 16, 3, 633, 133, 12, 6, 3, 1301, 459, 4, 1751, 209, 3, 10785, 7693, 308, 6, 676, 80, 32, 2137, 1110, 3008, 31, 1, 929, 4, 42, 5120, 469, 9, 2665, 1751, 1, 223, 55, 16, 54, 828, 1318, 847, 228, 9, 40, 96, 122, 1484, 57, 145, 36, 1, 996, 141, 27, 676, 122, 1, 13886, 411, 59, 94, 2278, 303, 772, 5, 3, 837, 11037, 20, 3, 1755, 646, 42, 125, 71, 22, 235, 101, 16, 46, 49, 624, 31, 702, 84, 702, 378, 3493, 12997, 2, 16816, 8422, 67, 27, 107, 3348]


In [ ]:
print(len(test_sequences[0]))
print(test_sequences[0])

164
[277, 171, 440, 11801, 44, 3318, 43, 3, 17, 15, 227, 1203, 71, 1668, 1209, 36, 1, 1301, 2016, 2225, 842, 4, 60, 47, 23, 52, 168, 10, 40, 119, 21, 456, 41, 98, 4, 1, 102, 88, 4, 175, 25, 2750, 8, 1, 4229, 2, 106, 23, 1704, 399, 20, 2, 92, 1547, 363, 73, 300, 31, 60, 55, 10, 119, 21, 456, 1, 106, 72, 141, 63, 456, 41, 6, 3, 52, 9290, 13323, 15894, 1, 436, 6, 26, 263, 122, 14, 550, 34, 1287, 237, 125, 71, 256, 331, 184, 87, 2, 284, 54, 4084, 4, 3, 4229, 24, 61, 12103, 735, 5, 27, 1573, 117, 11801, 414, 51, 72, 23, 70, 498, 1, 317, 93, 210, 4, 11, 4228, 11801, 713, 175, 29, 41, 2750, 72, 23, 576, 135, 15894, 6, 2163, 5, 27, 1, 115, 16, 54, 2593, 16291, 39, 12063, 54, 1233, 130, 9, 13, 29, 10, 97, 78, 5, 398, 36, 1583, 9, 122, 32, 531, 8]


In [ ]:
len(word_index)

88582

In [ ]:
for i, j in word_index.items():
  print(i, j)
  if j == 20:
    break

the 1
and 2
a 3
of 4
to 5
is 6
br 7
in 8
it 9
i 10
this 11
that 12
was 13
as 14
for 15
with 16
movie 17
but 18
film 19
on 20


In [ ]:
train_valid_data = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_valid_labels = tf.keras.utils.to_categorical(np.asarray(train_labels))
test_labels = tf.keras.utils.to_categorical(np.asarray(test_labels))

In [ ]:
print(train_valid_data.shape)

(25000, 1000)


In [ ]:
train_valid_data[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [ ]:
print(test_data.shape)

(25000, 1000)


In [ ]:
train_valid_labels.shape

(25000, 2)

In [ ]:
indices =np.arange(train_valid_data.shape[0])
np.random.shuffle(indices)
train_valid_data = train_valid_data[indices]
train_valid_labels = train_valid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * train_valid_data.shape[0])
print(num_validation_samples)

5000


In [ ]:
x_train = train_valid_data[:-num_validation_samples]
y_train = train_valid_labels[:-num_validation_samples]

x_test = train_valid_data[-num_validation_samples:]
y_test = train_valid_labels[-num_validation_samples:]

In [ ]:
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

In [ ]:
embeddings_index.get('attention')

array([-3.3414e-01,  4.6667e-01,  5.3744e-01,  5.7743e-02,  2.9642e-01,
        2.5224e-01, -6.5586e-01, -4.1668e-01,  2.1959e-01, -4.9413e-01,
       -2.1816e-01, -9.0227e-02, -3.5179e-02, -2.7279e-01, -1.2343e-01,
        1.6808e-01, -5.0623e-01, -4.0497e-01, -1.6763e-01,  4.9066e-01,
       -8.8020e-02, -1.2339e-01, -3.8436e-01, -2.7766e-01, -1.3403e-01,
        1.4342e-01, -2.9177e-01, -2.1146e-02,  5.2180e-01, -2.1213e-01,
        3.0860e-02,  1.0402e-01, -1.6807e-01,  4.6170e-01, -5.4806e-01,
       -6.6849e-02, -3.3180e-01,  3.7257e-01, -7.4962e-01,  6.2741e-01,
       -4.9500e-01, -4.0996e-01, -1.4686e-01, -2.7166e-01, -7.7093e-02,
       -2.8342e-01,  6.3663e-02, -1.5734e-01,  6.9649e-01, -9.6694e-01,
        4.4510e-01, -2.4521e-01, -4.8447e-01,  1.1957e+00,  2.9929e-02,
       -2.0425e+00, -2.8603e-01, -3.9043e-01,  1.2197e+00, -4.7760e-01,
       -2.1191e-02,  9.3080e-01, -1.8173e-01, -7.5721e-02,  1.1242e+00,
       -8.2276e-02,  5.7149e-02, -2.3585e-01,  3.5901e-01,  6.92

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
  if i > MAX_NUM_WORDS:
    continue
  
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = tf.keras.layers.Embedding(num_words, 
                                           EMBEDDING_DIM,
                                           embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                           input_length=MAX_SEQUENCE_LENGTH,
                                           trainable=False)

###1D CNN

In [ ]:
cnn_model = tf.keras.models.Sequential()
cnn_model.add(embedding_layer)
cnn_model.add(tf.keras.layers.Conv1D(128, 5, activation='relu'))
cnn_model.add(tf.keras.layers.MaxPool1D(5))
cnn_model.add(tf.keras.layers.Conv1D(128, 5, activation='relu'))
cnn_model.add(tf.keras.layers.MaxPool1D(5))
cnn_model.add(tf.keras.layers.Conv1D(128, 5, activation='relu'))
cnn_model.add(tf.keras.layers.GlobalMaxPooling1D())
cnn_model.add(tf.keras.layers.Dense(128, activation='relu'))
cnn_model.add(tf.keras.layers.Dense(len(labels_index), activation='relu'))

In [ ]:
cnn_model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])

In [ ]:
cnn_model.fit(x_train, y_train,
              batch_size=32,
              epochs=2,
              validation_data=(x_test, y_test))

Epoch 1/2
625/625 [==============================] - 173s 277ms/step - loss: nan - acc: 0.4975 - val_loss: nan - val_acc: 0.5100
Epoch 2/2
625/625 [==============================] - 171s 274ms/step - loss: nan - acc: 0.4975 - val_loss: nan - val_acc: 0.5100


In [ ]:
score, acc = cnn_model.evaluate(test_data, test_labels)
print(acc)

0.5


###LSTM Model

####With Our Own Embedding Layer

In [ ]:
rnn_model = tf.keras.models.Sequential()
rnn_model.add(tf.keras.layers.Embedding(MAX_NUM_WORDS, 128))
rnn_model.add(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnn_model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
rnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
rnn_model.fit(x_train, y_train,
              batch_size=32, epochs=1,
              validation_data=(x_test, y_test))

In [ ]:
score, acc = cnn_model.evaluate(test_data, test_labels)
print(acc)

####With Pre-trained Embedding Layer

In [ ]:
rnn_model = tf.keras.models.Sequential()
rnn_model.add(embedding_layer)
rnn_model.add(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnn_model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
rnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
rnn_model.fit(x_train, y_train,
              batch_size=32, epochs=1,
              validation_data=(x_test, y_test))

625/625 [==============================] - 1515s 2s/step - loss: 0.5762 - accuracy: 0.6996 - val_loss: 0.4493 - val_accuracy: 0.7978


In [ ]:
score, acc = rnn_model.evaluate(test_data, test_labels)
print(acc)

782/782 [==============================] - 244s 312ms/step - loss: 0.4492 - accuracy: 0.7976
0.7976400256156921


##BERT Sentiment Classification


######On IMDB Reviews Dataset [kaggle](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)


###With pytorch_pretrained_bert

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/'
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip \*.zip && rm *.zip

In [ ]:
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df = pd.read_csv("IMDB Dataset.csv",engine='python', error_bad_lines=False)

In [ ]:
df.tail(10)

,review,sentiment
49990,"Lame, lame, lame!!! A 90-minute cringe-fest th...",negative
49991,"Les Visiteurs, the first movie about the medie...",negative
49992,John Garfield plays a Marine who is blinded by...,positive
49993,Robert Colomb has two full-time jobs. He's kno...,negative
49994,This is your typical junk comedy.<br /><br />T...,negative
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
df['sentiment'] = preprocessing.LabelEncoder().fit_transform(df.sentiment)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [ ]:
def strip(text):
  soup = bs4.BeautifulSoup(text, 'html.parser')
  text = re.sub('\[[^]]*\]', '', soup.get_text())
  pattern = r"[^a-zA-Z0-9\s,']"
  text = re.sub(pattern, '', text)
  return text

In [ ]:
for i, j in enumerate(zip(df.review, temp)):
  if i == 5:
    break
  print(j[0], '\n Changes to=====>\n', j[1])
  print('\n')

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [ ]:
df['review'] = df.review.apply(strip)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tech...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,Petter Mattei's Love in the Time of Money is a...,1


BERT expects a special token [CLS] to indicate the start of the sentence and   a special token [SEP] to indicate end of it.

In [ ]:
sentence = [f'[CLS]{sen}[SEP]' for sen in df.review]

In [ ]:
tokenizer = BertTokenizer().from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = list(map(lambda t: tokenizer.tokenize(t)[:510] , sentence))
labels = list(df['sentiment'])

In [ ]:
MAX_LEN = 128
input_ids = tf.keras.sequences.pad_sequences(list(map(tokenizer.convert_tokens_to_ids, tokenized_texts)),
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = tf.keras.sequences.pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = model_selection.train_test_split(input_ids, labels, 
                                                            random_state=65, test_size=0.1)
train_masks, validation_masks, _, _ = model_selection.train_test_split(attention_masks, input_ids,
                                             random_state=65, test_size=0.1)
                                             
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

train_data = torch.utils.data.TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = torch.utils.data.RandomSampler(train_data)
train_dataloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=16)

validation_data = torch.utils.data.TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = torch.utils.data.SequentialSampler(validation_data)
validation_dataloader = torch.utils.data.DataLoader(validation_data, sampler=validation_sampler, batch_size=16)


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

torch.cuda.empty_cache() 
train_loss_set = []
epochs = 4

for u in tqdm.trange(epochs, desc="Epoch"):  
  
    model.train()  
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        train_loss_set.append(loss.item())    
        loss.backward()
        optimizer.step()
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
       

    model.eval()
    eval_accuracy = 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))


###With Ktrain

In [ ]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True)

IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

In [ ]:
(x_train, y_train), (x_test, y_test), preproc = ktrain.text.texts_from_folder(IMDB_DATADIR, 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 'test'],
                                                                       classes=['pos', 'neg'])

detected encoding: utf-8
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=32)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner.fit_onecycle(2e-5, 1)

#Interpert a Deep Learning Model

###Data

In [23]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.2

vocab_size = 20000  
maxlen = 1000 

In [3]:
#helper functions to lead the data
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)

    train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
    return train_df, test_df

In [4]:
if not os.path.exists('aclImdb'):
    train,test = download_and_load_datasets()
else:
    train = load_dataset('aclImdb/train')
    test = load_dataset('aclImdb/test')

84140032/84125825 [==============================] - 2s 0us/step


###Lime

In [7]:

train_texts = train['sentence'].values
train_labels = train['polarity'].values

test_texts = test['sentence'].values
test_labels = test['polarity'].values

labels_index = {'pos':1, 'neg':0} 

print(train.columns)

Index(['sentence', 'sentiment', 'polarity'], dtype='object')


In [10]:
class TextsToSequences( tf.keras.preprocessing.text.Tokenizer, base.BaseEstimator, pipeline.TransformerMixin):
    def __init__(self,  **kwargs):
        super().__init__(**kwargs)
        
    def fit(self, texts, y=None):
        self.fit_on_texts(texts)
        return self
    
    def transform(self, texts, y=None):
        return np.array(self.texts_to_sequences(texts))
        

class Padder(base.BaseEstimator, pipeline.TransformerMixin):
    def __init__(self, maxlen=500):
        self.maxlen = maxlen
        self.max_index = None
        
    def fit(self, X, y=None):
        self.max_index = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=self.maxlen).max()
        return self
    
    def transform(self, X, y=None):
        X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=self.maxlen)
        X[X > self.max_index] = 0
        return X

sequencer = TextsToSequences(num_words=vocab_size)
padder = Padder(maxlen)

In [20]:
batch_size = 64
max_features = vocab_size + 1

def create_model(max_features):
    rnnmodel = tf.keras.models.Sequential()
    rnnmodel.add(tf.keras.layers.Embedding(MAX_NUM_WORDS, 128))
    rnnmodel.add(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    rnnmodel.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return rnnmodel

sklearn_lstm = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=1, batch_size=32, 
                               max_features=max_features, verbose=1)

my_pipeline = pipeline.make_pipeline(sequencer, padder, sklearn_lstm)

my_pipeline.fit(train_texts, train_labels)

In [ ]:
y_preds = pipeline.predict(test_texts)

In [ ]:
100 * metrics.accuracy_score(y_preds, test_labels)

In [ ]:
idx = 20
text_sample = test_texts[idx]
class_names = ['negative', 'positive']

print('Sample {}: last 1000 words (only part used by the model)'.format(idx))
print('-'*50)
print(" ".join(text_sample.split()[-1000:]))
print('-'*50)
print('Probability(positive) =', pipeline.predict_proba([text_sample])[0,1])
print('True class: %s' % class_names[test_labels[idx]])

In [ ]:
import seaborn as sns
from collections import OrderedDict
from lime.lime_text import LimeTextExplainer

explainer = LimeTextExplainer(class_names=class_names)
explanation = explainer.explain_instance(text_sample, pipeline.predict_proba, num_features=10)

weights = OrderedDict(explanation.as_list())
lime_weights = pd.DataFrame({'words': list(weights.keys()), 'weights': list(weights.values())})

sns.barplot(x="words", y="weights", data=lime_weights);
plt.xticks(rotation=45)
plt.title('Sample {} features weights given by LIME'.format(idx));

###Shap

In [22]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

trainvalid_data = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
trainvalid_labels = tf.keras.utils.to_categorical(np.asarray(train_labels))
test_labels = tf.keras.utils.to_categorical(np.asarray(test_labels))

indices = np.arange(trainvalid_data.shape[0])
np.random.shuffle(indices)

trainvalid_data = trainvalid_data[indices]
trainvalid_labels = trainvalid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * trainvalid_data.shape[0])

x_train = trainvalid_data[:-num_validation_samples]
y_train = trainvalid_labels[:-num_validation_samples]
x_val = trainvalid_data[-num_validation_samples:]
y_val = trainvalid_labels[-num_validation_samples:]

Found 88582 unique tokens.


In [ ]:
def create_model(max_features):
    rnnmodel = tf.keras.models.Sequential()
    rnnmodel.add(tf.keras.layers.Embedding(MAX_NUM_WORDS, 128))
    rnnmodel.add(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    rnnmodel.add(tf.keras.layers.Dense(2, activation='sigmoid'))
    rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return rnnmodel

sklearn_lstm = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=1, batch_size=32, 
                               max_features=max_features, verbose=1)

my_pipeline = pipeline.make_pipeline(sequencer, padder, sklearn_lstm)

my_pipeline.fit(train_texts, train_labels)


batch_size = 64
max_features = vocab_size + 1

rnnmodel = tf.keras.models.Sequential()
rnnmodel.add(tf.keras.layers.Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(tf.keras.layers.Dense(2, activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

rnnmodel.fit(x_train, y_train,
          batch_size=32,
          epochs=2,
          validation_data=(x_val, y_val))
score, acc = rnnmodel.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

In [ ]:
shap.initjs()
explainer = shap.DeepExplainer(rnnmodel, x_train[:20])

shap_values = explainer.shap_values(x_val[:5])

words = tf.keras.datasets.imdb.get_word_index()
num2word = {}
for w in words.keys():
    num2word[words[w]] = w
x_val_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), x_val[i]))) for i in range(10)])

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], x_val_words[0])